In [6]:
import sys
sys.path.append('../')

import random
import requests
import urllib.parse
from bs4 import BeautifulSoup
from objects.profile import Profile
import json
from collections import OrderedDict

from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import pandas as pd
from utils import create_driver

from data_sources.google_scholar import scrape_publication_details, scrape_google_scholar_profile, save_google_scholar_profile, save_publications_details

In [2]:
with open('./data/individual_profiles.json', 'r') as f:
    ind_profile = json.load(f)
    
google_scholar_profile_url = []

for profile in ind_profile:
    if profile['google_scholar'] is not None:
        google_scholar_profile_url.append([profile['full_name'], profile['google_scholar']])
len(google_scholar_profile_url)

32

# Scraping Google Scholar Profile Page

In [ ]:
for profile in google_scholar_profile_url:
    name, url = profile[0], profile[1]

    g_profile = scrape_google_scholar_profile(url)
    save_google_scholar_profile(g_profile, output_filename=f'google_scholar_{name.lower().replace(" ","_")}.json')

    if random.uniform(0,1)>0.8:
        print('Sleeping')
        time.sleep(60)

# Scraping Citation Information

In [ ]:
driver = create_driver(debug=True)

for j, profile in enumerate(google_scholar_profile_url):
    name, url = profile[0], profile[1]
    with open(f'./data/google_scholar_{name.lower().replace(" ","_")}.json', 'r') as f:
        data = json.load(f)
    print(f'{j}, {name}')
    details = {}
    try:
        for i, publication in enumerate(data['publications']):
            key = publication['title_link']
            driver.get(key)
            time.sleep(random.uniform(3,6))
            details[key] = scrape_publication_details(key, driver)
            if random.uniform(0,1)>0.75:
                time.sleep(random.uniform(5,10))
            if random.uniform(0,1)>0.9:
                driver.quit()
                driver = create_driver(debug=False)
            
            if i%10==0:
                save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    except Exception as e:
        print(f'Stopped at {j} {name}')
        print(e)

        save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
        break
    
    
    save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    driver.quit()
    driver = create_driver(debug=False)

    time.sleep(random.uniform(3*60, 5*60))

# Re running any publications with error

In [20]:
from striprtf.striprtf import rtf_to_text

with open("problem_google_scholar_publication.rtf") as infile:
    content = infile.read()
    text = rtf_to_text(content)
text = text.split('\n')
faulty_url = []
for t in text:
    if not t.isnumeric() and t!='':
        faulty_url.append(t.split(' at ')[1])

In [ ]:
driver = create_driver(debug=True)

for j, profile in enumerate(google_scholar_profile_url[1:]):
    name, url = profile[0], profile[1]
    with open(f'./data/google_scholar_{name.lower().replace(" ","_")}.json', 'r') as f:
        data = json.load(f)
    details = {}
    print(len(faulty_url))
    for i, publication in enumerate(data['publications']):
        key = publication['title_link']
        if key in faulty_url:
            driver.get(key)
            time.sleep(random.uniform(3,6))
            details[key] = scrape_publication_details(key, driver)
            if random.uniform(0,1)>0.75:
                time.sleep(random.uniform(5,10))
            if random.uniform(0,1)>0.9:
                driver.quit()
                driver = create_driver(debug=False)
            faulty_url.remove(key)

        save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')

    if len(faulty_url)==0:
        break
driver.quit()